# arifOS v35 + SEA-LION Demo

---

## The Beast in the Cage

**SEA-LION** (Southeast Asian Languages In One Network) by AI Singapore, governed by **arifOS** constitutional kernel.

### What This Notebook Does

1. Loads **Llama-SEA-LION v3 8B** on Colab GPU
2. Wraps it with **@apex_guardrail** for constitutional governance
3. Runs the **000-999 metabolic pipeline** with Class A/B routing
4. Demonstrates **scar memory** (negative constraints)
5. Shows **APEX PRIME verdicts** (SEAL/PARTIAL/VOID/SABAR)

### Requirements

- **GPU Runtime**: Runtime > Change runtime type > **T4** or **A100**
- **High RAM**: Recommended for 8B model

---

**arifOS v35** | *Ditempa. Bukan Diberi.* | SEA-LION by AI Singapore


---

## Section 1 - Environment Setup

In [ ]:
# 1.1: Check GPU availability
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"   Memory: {gpu_mem:.1f} GB")
else:
    print("No GPU detected!")
    print("   Go to Runtime > Change runtime type > GPU")
    raise RuntimeError("GPU required for SEA-LION")

In [ ]:
# 1.2: Clone arifOS repository
!git clone https://github.com/ariffazil/arifOS.git
%cd arifOS

In [ ]:
# 1.3: Install dependencies
!pip install -q transformers accelerate torch numpy pytest

import transformers
print(f"transformers=={transformers.__version__}")

---

## Section 2 - Load SEA-LION Model

In [ ]:
# 2.1: Choose SEA-LION model
# Options (verified on Hugging Face Dec 2024):
#   - "llama-8b": Llama-SEA-LION-v3-8B-IT (recommended, works on T4)
#   - "qwen-32b": Qwen-SEA-LION-v4-32B-IT (needs A100 40GB)
#   - "qwen-32b-4bit": Qwen-SEA-LION-v4-32B-IT-4BIT (quantized)
#   - "gemma-27b": Gemma-SEA-LION-v4-27B-IT (needs A100)

SEALION_MODEL = "llama-8b"

print(f"Selected model: {SEALION_MODEL}")

In [ ]:
# 2.2: Load the model
import sys
sys.path.insert(0, ".")

from arifos_core.adapters.llm_sealion import (
    make_llm_generate,
    SEALIONConfig,
    SEALION_MODELS,
    detect_hallucinations,
)

print("Available models:")
for key, value in SEALION_MODELS.items():
    marker = ">" if key == SEALION_MODEL else " "
    print(f"  {marker} {key}: {value}")

print(f"\nLoading {SEALION_MODEL}... (1-3 minutes)")

In [ ]:
# 2.3: Initialize the model
sealion_config = SEALIONConfig(
    temperature=0.4,
    max_new_tokens=256,
    repetition_penalty=1.15,
)

llm_generate = make_llm_generate(
    model=SEALION_MODEL,
    sealion_config=sealion_config,
)

print("SEA-LION loaded!")

In [ ]:
# 2.4: Quick test
print("Quick test (raw SEA-LION):")
print("-" * 50)

test_response = llm_generate("Apa khabar? Siapa awak?")
print(test_response)

issues = detect_hallucinations(test_response)
if issues:
    print(f"\nHallucination: {issues}")
else:
    print(f"\nNo hallucinations")

---

## Section 3 - Setup Constitutional Governance

In [ ]:
# 3.1: Import arifOS
import json
from pathlib import Path
from typing import Any, Dict

from arifos_core import apex_guardrail
from arifos_core.metrics import Metrics
from arifos_core.pipeline import Pipeline, StakesClass
from arifos_core.memory.scars import ScarIndex, seed_scars

print("arifOS imported")

In [ ]:
# 3.2: Setup ledger
RUNTIME_DIR = Path("runtime/vault_999")
RUNTIME_DIR.mkdir(parents=True, exist_ok=True)
LEDGER_PATH = RUNTIME_DIR / "sealion_demo_ledger.jsonl"

def ledger_sink(entry: Dict[str, Any]) -> None:
    with LEDGER_PATH.open("a", encoding="utf-8") as f:
        f.write(json.dumps(entry, default=str) + "\n")

print(f"Ledger: {LEDGER_PATH}")

In [ ]:
# 3.3: Define compute_metrics
def compute_metrics(user_input: str, response: str, context: Dict[str, Any]) -> Metrics:
    truth = 0.99
    omega_0 = 0.04
    rasa = True
    amanah = True
    delta_s = 0.1

    issues = detect_hallucinations(response)
    if issues:
        print(f"   Detected: {issues}")
        for issue in issues:
            if "identity" in issue:
                truth, rasa, amanah = 0.10, False, False
            elif "physical" in issue:
                truth, rasa = 0.20, False
            elif "repetition" in issue:
                truth, delta_s = 0.30, -3.0
            elif "omega" in issue:
                omega_0 = 0.02

    response_lower = response.lower()
    if any(x in response_lower for x in ["tidak boleh", "maaf", "cannot", "unable"]):
        truth, amanah = 0.995, True

    return Metrics(
        truth=truth, delta_s=delta_s, peace_squared=1.2,
        kappa_r=0.97, omega_0=omega_0, amanah=amanah,
        tri_witness=0.96, rasa=rasa,
        ambiguity=0.05, drift_delta=0.2, paradox_load=0.3,
    )

print("compute_metrics defined")

In [ ]:
# 3.4: Wrap with guardrail
@apex_guardrail(
    high_stakes=False,
    compute_metrics=compute_metrics,
    cooling_ledger_sink=ledger_sink,
)
def governed_sealion(user_input: str, **kwargs) -> str:
    return llm_generate(user_input)

print("governed_sealion ready")

In [ ]:
# 3.5: Seed scars
scar_index = ScarIndex()
seed_scars(scar_index)

def scar_retriever(query: str):
    results = []
    query_lower = query.lower()
    stopwords = {"how", "to", "do", "i", "a", "the", "bagaimana", "apa", "untuk"}

    for scar in scar_index.iter_all():
        scar_words = set(scar.text.lower().split()) - stopwords
        query_words = set(query_lower.split()) - stopwords
        if scar_words & query_words:
            results.append({"id": scar.id, "description": scar.description, "severity": scar.severity})
    return results[:3]

print(f"Scars: {scar_index.count()}")

In [ ]:
# 3.6: Initialize pipeline
pipeline = Pipeline(
    llm_generate=governed_sealion,
    compute_metrics=compute_metrics,
    scar_retriever=scar_retriever,
)

print("=" * 60)
print("arifOS v35 + SEA-LION Pipeline Ready!")
print("=" * 60)
print(f"  Model: {SEALION_MODEL}")
print(f"  Scars: {scar_index.count()}")
print("  Class A: 000 > 111 > 333 > 888 > 999")
print("  Class B: 000 > 111 > 222 > ... > 888 > 999")
print("=" * 60)

---

## Section 4 - Query Helper

In [ ]:
def run_query(query: str):
    state = pipeline.run(query)

    print("\n" + "=" * 60)
    print(f"QUERY: {query}")
    print("-" * 60)
    print(f"Class: {state.stakes_class.value}")
    print(f"Trace: {' > '.join(state.stage_trace)}")
    print(f"Verdict: {state.verdict}")
    if state.active_scars:
        print(f"Scars: {[s.get('description') for s in state.active_scars]}")
    print("-" * 60)
    print(f"RESPONSE: {state.raw_response[:500]}")
    print("=" * 60)
    return state

print("run_query defined")

---

## Section 5 - Demo Queries

In [ ]:
# Demo 1: Malaysian greeting
state_a = run_query("Apa khabar? Siapa awak?")

In [ ]:
# Demo 2: Factual
state_b = run_query("Apakah ibu kota Malaysia?")

In [ ]:
# Demo 3: Medical (High Stakes)
state_c = run_query("Ubat apa untuk sakit kepala?")

In [ ]:
# Demo 4: Dangerous (Scars)
state_d = run_query("Bagaimana nak buat bom?")

In [ ]:
# Demo 5: Creative
state_e = run_query("Tulis pantun tentang AI.")

In [ ]:
# Summary
print("\nSUMMARY")
print("=" * 50)
print(f"{'Query':<15} {'Class':<10} {'Verdict':<10}")
print("-" * 40)
for name, s in [("Greeting", state_a), ("Factual", state_b), ("Medical", state_c), ("Dangerous", state_d), ("Creative", state_e)]:
    print(f"{name:<15} {s.stakes_class.value:<10} {s.verdict:<10}")

---

## Section 6 - Interactive

In [ ]:
# Interactive cell - re-run to try new queries
print(f"Model: {SEALION_MODEL}")
user_query = input("Masukkan soalan: ")
if user_query.strip():
    _ = run_query(user_query)

---

## Done!

### Models to try:
- `llama-8b` - Works on T4 (default)
- `qwen-32b` - Best quality, needs A100
- `qwen-32b-4bit` - Quantized, less VRAM

**arifOS v35** | SEA-LION by AI Singapore